In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE
from collections import Counter

In [3]:
!pip install scikit-learn

In [2]:
credit_card = pd.read_csv("creditcard.csv")
credit_card.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [40]:
credit_card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [6]:
credit_card.shape

(284807, 31)

In [8]:
label = ['not fraud', 'fraud']

In [9]:
# imbalanved data
credit_card['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [55]:
# Independent and dependent vaiable
X = credit_card.drop('Class', axis = 1)
y = credit_card.Class
y

0         0
1         0
2         0
3         0
4         0
         ..
284802    0
284803    0
284804    0
284805    0
284806    0
Name: Class, Length: 284807, dtype: int64

In [49]:
# Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=28)

In [14]:
# Logiistic Regression
# Random Forest (robust to imbalanced data)

In [15]:
logistic_model = LogisticRegression()
grid = {'C':10.0**np.arange(-2,3)}
cv = KFold(n_splits=5, shuffle=False)

In [20]:
clf = GridSearchCV(logistic_model, grid, scoring='accuracy', cv = cv, n_jobs=2)
clf.fit(X_train,y_train)

C:\Users\Vikitha Reddy\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=2,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             scoring='accuracy')

In [25]:
clf.best_params_, clf.best_score_

({'C': 0.1}, 0.998946650348001)

In [23]:
# prediction on testing data
y_pred = clf.predict(X_test)
print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[85255    31]
 [   50   107]]
0.9990519995786665
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85286
           1       0.78      0.68      0.73       157

    accuracy                           1.00     85443
   macro avg       0.89      0.84      0.86     85443
weighted avg       1.00      1.00      1.00     85443



In [26]:
# TP: 85255- increase :1 predicted as 1 
# TN: 107 - increase : 0 prediced as 0
# FN: 31 - decrease this value : misclassifications : 1 predicted as 0
# FP: 50 - decrese this value: misclassifications: 0 predicted as 1
# Recall: TP/TP+FN : 0: 1.00: 1 predicted as 1
#                  : 1: 0.68
# precision: TP/TP+FP : 0: 1.00
#                     : 1: 0.78
# F1-score : 0: 1.00; 1: 0.73

In [29]:
# Random Forest
RF_model = RandomForestClassifier()
RF_model.fit(X_train, y_train)

RandomForestClassifier()

In [30]:
# prediction on testing data
y_pred = RF_model.predict(X_test)
print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[85295    21]
 [   10   117]]
0.9996371850239341
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85316
           1       0.85      0.92      0.88       127

    accuracy                           1.00     85443
   macro avg       0.92      0.96      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [32]:
# Logistic Regression
# TP: 85255- 85295
# TN: 107
# FN: 31 
# FP: 50 
# Recall: TP/TP+FN : 0: 1.00  ;  1: 0.68 
# precision: TP/TP+FP : 0: 1.00 ; 1: 0.78
# F1-score : 0: 1.00 ; 1: 0.73 

# Random Forest
# TP: 85295
# TN: 117
# FN: 21
# FP: 10
# Recall: 0: 1.00;  1: 0.92
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88

In [17]:
# UnderSampling
under_sampling = NearMiss(0.8)
X_train_us, y_train_us = under_sampling.fit_resample(X_train, y_train)
print("The number of classes before under_sampling{}".format(Counter(y_train)))
print("The number of classes before under_sampling{}".format(Counter(y_train_us)))

C:\Users\Vikitha Reddy\anaconda3\lib\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


The number of classes before under_samplingCounter({0: 199010, 1: 354})
The number of classes before under_samplingCounter({0: 442, 1: 354})


In [18]:
# undersampling
RF_model = RandomForestClassifier()
RF_model.fit(X_train_us, y_train_us)

RandomForestClassifier()

In [19]:
# prediction on testing data
y_pred = RF_model.predict(X_test)
print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[61940     5]
 [23365   133]]
0.7264843228819213
              precision    recall  f1-score   support

           0       0.73      1.00      0.84     61945
           1       0.96      0.01      0.01     23498

    accuracy                           0.73     85443
   macro avg       0.84      0.50      0.43     85443
weighted avg       0.79      0.73      0.61     85443



In [ ]:
# TP: 85295
# TN: 117
# FN: 21
# FP: 10
# Recall: 0: 1.00;  1: 0.92
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88

# undersampling
# TP: 61940
# TN: 133
# FN: 5
# FP: 23365
# Recall: 0: 1.00;  1: 0.01
# precision : 0: 0.73; 1: 0.96
# F1-score : 0: 0.84 ; 1: 0.01

In [ ]:
# balance the data  using oversampling (Random, SMOTE)

In [21]:
# OverSampling
os = RandomOverSampler()
X_train_os, y_train_os = os.fit_resample(X_train, y_train)
print("The number of classes before under_sampling{}".format(Counter(y_train)))
print("The number of classes before under_sampling{}".format(Counter(y_train_os)))

The number of classes before under_samplingCounter({0: 199010, 1: 354})
The number of classes before under_samplingCounter({0: 199010, 1: 199010})


In [22]:
# over sampling
RF_model = RandomForestClassifier()
RF_model.fit(X_train_os, y_train_os)

RandomForestClassifier()

In [23]:
# prediction on testing data
y_pred = RF_model.predict(X_test)
print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[85293    21]
 [   12   117]]
0.9996137776061234
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85314
           1       0.85      0.91      0.88       129

    accuracy                           1.00     85443
   macro avg       0.92      0.95      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
# TP: 85295
# TN: 117
# FN: 21
# FP: 10
# Recall: 0: 1.00;  1: 0.92
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88

# TP: 85293
# TN: 117
# FN: 21
# FP: 12
# Recall: 0: 1.00;  1: 0.91
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88

In [24]:
# OverSampling
os = RandomOverSampler(0.75)
X_train_os, y_train_os = os.fit_resample(X_train, y_train)
print("The number of classes before under_sampling{}".format(Counter(y_train)))
print("The number of classes before under_sampling{}".format(Counter(y_train_os)))

C:\Users\Vikitha Reddy\anaconda3\lib\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=0.75 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


The number of classes before under_samplingCounter({0: 199010, 1: 354})
The number of classes before under_samplingCounter({0: 199010, 1: 149257})


In [25]:
# over sampling
RF_model = RandomForestClassifier()
RF_model.fit(X_train_os, y_train_os)

RandomForestClassifier()

In [26]:
# prediction on testing data
y_pred = RF_model.predict(X_test)
print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[85295    21]
 [   10   117]]
0.9996371850239341
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85316
           1       0.85      0.92      0.88       127

    accuracy                           1.00     85443
   macro avg       0.92      0.96      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [27]:
# before balancing
# TP: 85295
# TN: 117
# FN: 21
# FP: 10
# Recall: 0: 1.00;  1: 0.92
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88

# oversampling where sampling_strategy = default
# TP: 85293
# TN: 117
# FN: 21
# FP: 12
# Recall: 0: 1.00;  1: 0.91
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88

# oversampling where sampling_strategy = 0.75
# TP: 85293
# TN: 117
# FN: 21
# FP: 10
# Recall: 0: 1.00;  1: 0.92
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88

In [28]:
sm = SMOTE(0.75)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
print("The number of classes before under_sampling{}".format(Counter(y_train)))
print("The number of classes before under_sampling{}".format(Counter(y_train_sm)))

C:\Users\Vikitha Reddy\anaconda3\lib\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=0.75 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


The number of classes before under_samplingCounter({0: 199010, 1: 354})
The number of classes before under_samplingCounter({0: 199010, 1: 149257})


In [30]:
# SMOTE
RF_model = RandomForestClassifier()
RF_model.fit(X_train_sm, y_train_sm)

RandomForestClassifier()

In [31]:
# prediction on testing data
y_pred = RF_model.predict(X_test)
print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[85284    14]
 [   21   124]]
0.9995903701883126
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85298
           1       0.90      0.86      0.88       145

    accuracy                           1.00     85443
   macro avg       0.95      0.93      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
# oversampling where sampling_strategy = 0.75
# TP: 85293
# TN: 117
# FN: 21
# FP: 10
# Recall: 0: 1.00;  1: 0.92
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88

# SMOTE
# TP: 85284
# TN: 124
# FN: 14
# FP: 21
# Recall: 0: 1.00;  1: 0.86
# precision : 0: 1.00; 1: 0.90
# F1-score : 0: 1.00 ; 1: 0.88

In [32]:
# oversampling is performing better than SMOTE

In [33]:
credit_card.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [58]:
# # dropping the time column

In [56]:
X = X.drop('Time', axis = 1)
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [59]:
# Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=28)

In [60]:
# OverSampling
os = RandomOverSampler(0.75)
X_train_os, y_train_os = os.fit_resample(X_train, y_train)
print("The number of classes before under_sampling{}".format(Counter(y_train)))
print("The number of classes before under_sampling{}".format(Counter(y_train_os)))

C:\Users\Vikitha Reddy\anaconda3\lib\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=0.75 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


The number of classes before under_samplingCounter({0: 199010, 1: 354})
The number of classes before under_samplingCounter({0: 199010, 1: 149257})


In [61]:
# over sampling
RF_model = RandomForestClassifier()
RF_model.fit(X_train_os, y_train_os)

RandomForestClassifier()

In [62]:
# prediction on testing data
y_pred = RF_model.predict(X_test)
print(confusion_matrix(y_pred, y_test))
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[85294    21]
 [   11   117]]
0.9996254813150287
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85315
           1       0.85      0.91      0.88       128

    accuracy                           1.00     85443
   macro avg       0.92      0.96      0.94     85443
weighted avg       1.00      1.00      1.00     85443



In [ ]:
# oversampling where sampling_strategy = 0.75
# TP: 85293
# TN: 117
# FN: 21
# FP: 10
# Recall: 0: 1.00;  1: 0.92
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88

# SMOTE
# TP: 85284
# TN: 124
# FN: 14
# FP: 21
# Recall: 0: 1.00;  1: 0.86
# precision : 0: 1.00; 1: 0.90
# F1-score : 0: 1.00 ; 1: 0.88

# oversampling where sampling_strategy = 0.75 after dropping TIME column
# TP: 85294
# TN: 117
# FN: 21
# FP: 11
# Recall: 0: 1.00;  1: 0.91
# precision : 0: 1.00; 1: 0.85
# F1-score : 0: 1.00 ; 1: 0.88